# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


### **Import Libraries**

In [ ]:
import requests
import pandas as pd
import numpy as np 
import datetime

In [ ]:
def getBoosterVersion(data): 
    for x in data["rocket"]: 
        if x: 
            response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
            BoosterVersion.append(response["name"])

In [ ]:
def getLaunchSite(data):
    for x in data["launchpad"]:
        if x:
            response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
            Longitude.append(response["longitude"])
            Latitude.append(response["latitude"])
            LaunchSite.append(response["name"])

In [ ]:
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

In [ ]:
def getCoreData(data): 
    for core in data["cores"]: 
        if core["core"] != None: 
            response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
            Block.append(response["block"])
            ReusedCount.append(response["reuse_count"])
            Serial.append(response["serial"])
        
        else: 
            Block.append(None)
            ReusedCount.append(None)
            Serial.append(None)
        
        Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
        Flights.append(core['flight'])
        GridFins.append(core['gridfins'])
        Reused.append(core['reused'])
        Legs.append(core['legs'])
        LandingPad.append(core['landpad'])

In [ ]:
url = "https://api.spacexdata.com/v4/launches/past"


In [ ]:
response = requests.get(url)

In [ ]:
# response.content

In [ ]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

In [ ]:
response.status_code

In [ ]:
data = pd.json_normalize(response.json())
data.head()

In [ ]:
# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
data['date'] = pd.to_datetime(data['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
data = data[data['date'] <= datetime.date(2020, 11, 13)]

In [ ]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

In [ ]:
BoosterVersion

In [ ]:
getBoosterVersion(data)

In [ ]:
BoosterVersion[0:5]

In [ ]:
getLaunchSite(data)

In [ ]:
getPayloadData(data)

In [ ]:
getCoreData(data)

In [ ]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

In [ ]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
df.head()


In [ ]:
data_falcon9 = df[df["BoosterVersion"] != "Falcon 1"]

In [ ]:
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
data_falcon9

In [ ]:
data_falcon9.isnull().sum()

In [ ]:
mean = data_falcon9["PayloadMass"].mean()

In [ ]:
data_falcon9["PayloadMass"].replace(np.nan, mean, inplace=True)

In [ ]:
data_falcon9.to_csv("dataset_part_1.csv", index = False)